In [1]:
import os
import sys

module_path = os.path.abspath("..")
if module_path not in sys.path:
    sys.path.append(module_path)

from utils import set_random_seed
set_random_seed()
    
import tensorflow as tf
from tensorflow.keras import backend as K
from layers.interaction import ExactInteractions

import numpy as np


## Exact layer 

Creates a interaction matrix S were each entry $i$ and $j$ are defined by:

$s_{ij}=\begin{cases}1 & q_i = d_i\\0 & otherwise\end{cases}$

In [2]:
   
K.clear_session()

input_query = tf.keras.layers.Input((10,), dtype="int32")
input_sentence = tf.keras.layers.Input((12,), dtype="int32")

exact_interaction = ExactInteractions()

_out = exact_interaction([input_query, input_sentence])

model = tf.keras.models.Model(inputs=[input_query, input_sentence], outputs=_out)
model.summary()
    

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 12)]         0                                            
__________________________________________________________________________________________________
exact_interactions (ExactIntera (None, 10, 12)       0           input_1[0][0]                    
                                                                 input_2[0][0]                    
Total params: 0
Trainable params: 0
Non-trainable params: 0
__________________________________________________________________________________________________


In [3]:
query = np.random.randint(1,4,(1, 10))
document = np.random.randint(1,4,(1, 12))

In [4]:
y = model.predict([query, document])
y

array([[[1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1.],
        [0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0.],
        [1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]]], dtype=float32)

In [5]:
# add padding
query[:,5:] = np.zeros(5,)
document[:,6:] = np.zeros(6,)
y = model.predict([query, document])
y

array([[[1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]], dtype=float32)

# Exact Layer with term importance

In [6]:
   
K.clear_session()

input_query = tf.keras.layers.Input((8,), dtype="int32")
input_sentence = tf.keras.layers.Input((4,), dtype="int32")

input_query_t_importance = tf.keras.layers.Input((8,), dtype="float32")
input_sentence_t_importance = tf.keras.layers.Input((4,), dtype="float32")

_inputs = [input_query, input_sentence, input_query_t_importance, input_sentence_t_importance]

exact_interaction = ExactInteractions()

_out = exact_interaction(_inputs)

model = tf.keras.models.Model(inputs=_inputs, outputs=_out)
model.summary()
    

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 4)]          0                                            
______________________________________________________________________________________________

In [7]:
query = np.random.randint(1,10,(1, 8))
sentence = np.random.randint(1,10,(1, 4))

query_importance = np.random.random((1, 8))
sentence_importance = np.random.random((1, 4))

In [8]:
y = model.predict([query, sentence, query_importance, sentence_importance])
print(y[0,:,:,0])
print(y[0,:,:,1])
print(y[0,:,:,2])

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 1. 0. 0.]]
[[0.45049927 0.45049927 0.45049927 0.45049927]
 [0.01326496 0.01326496 0.01326496 0.01326496]
 [0.94220173 0.94220173 0.94220173 0.94220173]
 [0.5632882  0.5632882  0.5632882  0.5632882 ]
 [0.3854165  0.3854165  0.3854165  0.3854165 ]
 [0.01596625 0.01596625 0.01596625 0.01596625]
 [0.23089382 0.23089382 0.23089382 0.23089382]
 [0.24102546 0.24102546 0.24102546 0.24102546]]
[[0.68326354 0.6099967  0.8331949  0.17336465]
 [0.68326354 0.6099967  0.8331949  0.17336465]
 [0.68326354 0.6099967  0.8331949  0.17336465]
 [0.68326354 0.6099967  0.8331949  0.17336465]
 [0.68326354 0.6099967  0.8331949  0.17336465]
 [0.68326354 0.6099967  0.8331949  0.17336465]
 [0.68326354 0.6099967  0.8331949  0.17336465]
 [0.68326354 0.6099967  0.8331949  0.17336465]]


In [9]:
query[:,4:] = np.zeros(4,)
sentence[:,2:] = np.zeros(2,)
query_importance[:,4:] = np.zeros(4,)
sentence_importance[:,2:] = np.zeros(2,)

y = model.predict([query, sentence, query_importance, sentence_importance])
print(y[0,:,:,0])
print(y[0,:,:,1])
print(y[0,:,:,2])

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0.45049927 0.45049927 0.         0.        ]
 [0.01326496 0.01326496 0.         0.        ]
 [0.94220173 0.94220173 0.         0.        ]
 [0.5632882  0.5632882  0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]]
[[0.68326354 0.6099967  0.         0.        ]
 [0.68326354 0.6099967  0.         0.        ]
 [0.68326354 0.6099967  0.         0.        ]
 [0.68326354 0.6099967  0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]]


# Semantic Layer

In [10]:
from interactions import SemanticInteractions

ImportError: No module named 'interactions'

In [11]:
from nir.embeddings import FastText
from nir.tokenizers import Regex

cache_folder = "/backup/IR"
prefix_name = "disk4_5"

# load tokenizer
tk = Regex.load_from_json(cache_folder=cache_folder, prefix_name=prefix_name)

# load embedding matrix
ft = FastText.maybe_load(cache_folder = cache_folder,
                         prefix_name = prefix_name,
                         path = "/backup/pre-trained_embeddings/fasttext/wiki.en.bin",
                         tokenizer = tk)

emb_matrix = ft.embedding_matrix()

DEBUG created tokenizer disk4_5_RegexTokenizer
False False
[LOAD FROM CACHE] Load embedding matrix from /backup/IR/embedding_wiki_disk4_5_RegexTokenizer


In [12]:

K.clear_session()

input_query = tf.keras.layers.Input((8,), dtype="int32")
input_sentence = tf.keras.layers.Input((4,), dtype="int32")

semantic_interaction = SemanticInteractions(emb_matrix)

_out = semantic_interaction([input_query, input_sentence])

model = tf.keras.models.Model(inputs=[input_query, input_sentence], outputs=_out)
model.summary()
    

[EMBEDDING MATRIX SHAPE] (228107, 300)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
semantic_interactions (Semantic (None, 8, 4, 3)      600         input_1[0][0]                    
                                                                 input_2[0][0]                    
Total params: 600
Trainable params: 600
Non-trainable params: 0
__________________________________________________________________________________________

In [13]:
query = np.random.randint(1,100000,(1, 8))
document = np.random.randint(1,100000,(1, 4))

In [14]:
y = model.predict([query, document])
print(y[0,:,:,0])
print(y[0,:,:,1])
print(y[0,:,:,2])

[[ 0.02600097  0.1523608   0.05045402  0.02214804]
 [-0.08191848  0.22749323  0.07356641  0.13409927]
 [ 0.16650498  0.00378791  0.17630842  0.17259032]
 [-0.02516811  0.07732472  0.17895198  0.17647892]
 [-0.02122113  0.21902236  0.1314221   0.10899112]
 [-0.17898647  0.2909802   0.2858552   0.18254572]
 [ 0.06219951  0.04914713  0.06359807  0.0281321 ]
 [ 0.03048387  0.15481643  0.07244178  0.12701723]]
[[ 0.07625329  0.07625329  0.07625329  0.07625329]
 [ 0.06301482  0.06301482  0.06301482  0.06301482]
 [ 0.1634911   0.1634911   0.1634911   0.1634911 ]
 [-0.01706506 -0.01706506 -0.01706506 -0.01706506]
 [ 0.10522851  0.10522851  0.10522851  0.10522851]
 [-0.13724078 -0.13724078 -0.13724078 -0.13724078]
 [ 0.08696583  0.08696583  0.08696583  0.08696583]
 [ 0.07390243  0.07390243  0.07390243  0.07390243]]
[[-0.09235676  0.03288544 -0.07960127  0.06511894]
 [-0.09235676  0.03288544 -0.07960127  0.06511894]
 [-0.09235676  0.03288544 -0.07960127  0.06511894]
 [-0.09235676  0.03288544 -0.

In [15]:
# add padding
query[:,4:] = np.zeros(4,)
document[:,2:] = np.zeros(2,)
y = model.predict([query, document])
print(y[0,:,:,0])
print(y[0,:,:,1])
print(y[0,:,:,2])

[[ 0.02600097  0.1523608  -0.         -0.        ]
 [-0.08191848  0.22749323 -0.         -0.        ]
 [ 0.16650498  0.00378791  0.          0.        ]
 [-0.02516811  0.07732472 -0.         -0.        ]
 [-0.         -0.          0.          0.        ]
 [-0.         -0.          0.          0.        ]
 [-0.         -0.          0.          0.        ]
 [-0.         -0.          0.          0.        ]]
[[ 0.07625329  0.07625329  0.          0.        ]
 [ 0.06301482  0.06301482  0.          0.        ]
 [ 0.1634911   0.1634911   0.          0.        ]
 [-0.01706506 -0.01706506 -0.         -0.        ]
 [-0.         -0.         -0.         -0.        ]
 [-0.         -0.         -0.         -0.        ]
 [-0.         -0.         -0.         -0.        ]
 [-0.         -0.         -0.         -0.        ]]
[[-0.09235676  0.03288544  0.          0.        ]
 [-0.09235676  0.03288544  0.          0.        ]
 [-0.09235676  0.03288544  0.          0.        ]
 [-0.09235676  0.03288544  0.

# Context semantic layer

In [16]:
from bert_wrapper import load_bert_model
from interactions import ContextedSemanticInteractions


In [17]:
K.clear_session()

input_query = tf.keras.layers.Input((20,), dtype="int32")
input_sentence = tf.keras.layers.Input((41,), dtype="int32")

bert_model = load_bert_model("albert_base", 64)

semantic_interaction = ContextedSemanticInteractions(context_embedding_layer = bert_model,
                                                     learn_term_weights = True,
                                                     cls_token_id = 2,
                                                     sep_token_id = 3,
                                                     pad_token_id = 0)

_out = semantic_interaction([input_query, input_sentence])

model = tf.keras.models.Model(inputs=[input_query, input_sentence], outputs=_out)
model.summary()

Fetching ALBERT model: albert_base version: 2
Already  fetched:  albert_base.tar.gz
already unpacked at: .models/albert_base
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Done loading 22 BERT weights from: .models/albert_base into <bert.model.BertModelLayer object at 0x7f7b95f19f60> (prefix:albert_base). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from saved model: 
	bert/embeddings/token_type_embeddings
	bert/pooler/dense/bias
	bert/pooler/dense/kernel
	cls/predictions/output_bias
	cls/predictions/transform/LayerNorm/beta
	cls/predictions/transform/LayerNorm/gamma
	cls/predictions/transform/dense/bias
	cls/predictions/transform/dense/kernel
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)          